## Network Flow Optimization Examples

In this tutorial, we want to solve the following example:

<img src="https://github.com/amirkfard/CVL609/blob/main/img/mip_81.webp?raw=1\" width="700">
<img src="https://github.com/amirkfard/CVL609/blob/main/img/mip_82.webp?raw=1\" width="700">
<img src="https://github.com/amirkfard/CVL609/blob/main/img/mip_83.webp?raw=1\" width="700">
<img src="https://github.com/amirkfard/CVL609/blob/main/img/mip_84.webp?raw=1\" width="700">

### Install OR-Tools

In [ ]:
pip install --upgrade --user ortools

### Declare the solver
In any MIP program, you start by importing the linear solver wrapper and declaring the MIP solver, as shown in the previous MIP example.

In [2]:
from ortools.linear_solver import pywraplp

### Create the data
The following code creates arrays containing the data for the example: the variable coefficients for the constraints and objective function, and bounds for the constraints.

In [64]:
import pandas as pd

Sources = ['W','M','H']
Destinations = ['E','T','O']
Capacities = {'W' : 100, 'M' : 300, 'H' : 300}
Requirements = {'E' : 300, 'T' : 200, 'O' : 200}
Costs_Values = [[5,4,3],[3,2,1],[9,7,5]]
Costs = pd.DataFrame(Costs_Values , columns = Destinations , index = Sources)
print(Costs)

   E  T  O
W  5  4  3
M  3  2  1
H  9  7  5


### Instantiate the solver
The following code instantiates the solver.

In [66]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

### Define the variables
The following code defines the variables for the example in a loop. For large problems, this is easier than defining the variables individually, as in the previous example.

In [67]:
infinity = solver.infinity()
x = {}
for s in Sources:
    for d in Destinations:
        x[s,d] = solver.IntVar(0, infinity, f'x[{s},{d}]')

print('Number of variables =', solver.NumVariables())

Number of variables = 9


### Define the constraints

The following code creates the constraints for the example, using loop.


In [68]:
for s in Sources:
    constraint_expr = []
    for d in Destinations:
        constraint_expr.append(x[s,d])
    solver.Add(sum(constraint_expr) == Capacities[s])

for d in Destinations:
    constraint_expr = []
    for s in Sources:
        constraint_expr.append(x[s,d])
    solver.Add(sum(constraint_expr) == Requirements[d])
    
print('Number of constraints =', solver.NumConstraints())

Number of constraints = 6


### Define the objective
The following code defines the objective function for the example.

In [69]:
obj_expr = []
for s in Sources:
    for d in Destinations:
        obj_expr.append(Costs[d][s] * x[s,d])
solver.Minimize(solver.Sum(obj_expr))

### Call the solver
The following code calls the solver.

In [70]:
status = solver.Solve()

### Display the solution
The following code displays the solution.

In [71]:
if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    for s in Sources:
        for d in Destinations:
            print(x[s,d].name(), ' = ', x[s,d].solution_value())
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
else:
    print('The problem does not have an optimal solution.')

Objective value = 3000.0
x[W,E]  =  100.0
x[W,T]  =  0.0
x[W,O]  =  0.0
x[M,E]  =  200.0
x[M,T]  =  100.0
x[M,O]  =  0.0
x[H,E]  =  0.0
x[H,T]  =  100.0
x[H,O]  =  200.0

Problem solved in 133958.000000 milliseconds
Problem solved in 6 iterations
Problem solved in 1 branch-and-bound nodes


### Complete programs
Here are the complete programs.

In [72]:
solver = pywraplp.Solver.CreateSolver('SCIP')

infinity = solver.infinity()
x = {}
for s in Sources:
    for d in Destinations:
        x[s,d] = solver.IntVar(0, infinity, f'x[{s},{d}]')

print('Number of variables =', solver.NumVariables())

for s in Sources:
    constraint_expr = []
    for d in Destinations:
        constraint_expr.append(x[s,d])
    solver.Add(sum(constraint_expr) == Capacities[s])

for d in Destinations:
    constraint_expr = []
    for s in Sources:
        constraint_expr.append(x[s,d])
    solver.Add(sum(constraint_expr) == Requirements[d])
    
print('Number of constraints =', solver.NumConstraints())

obj_expr = []
for s in Sources:
    for d in Destinations:
        obj_expr.append(Costs[d][s] * x[s,d])
solver.Minimize(solver.Sum(obj_expr))

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    for s in Sources:
        for d in Destinations:
            print(x[s,d].name(), ' = ', x[s,d].solution_value())
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
else:
    print('The problem does not have an optimal solution.')

Number of variables = 9
Number of constraints = 6
Objective value = 3000.0
x[W,E]  =  100.0
x[W,T]  =  0.0
x[W,O]  =  0.0
x[M,E]  =  200.0
x[M,T]  =  100.0
x[M,O]  =  0.0
x[H,E]  =  0.0
x[H,T]  =  100.0
x[H,O]  =  200.0

Problem solved in 11.000000 milliseconds
Problem solved in 6 iterations
Problem solved in 1 branch-and-bound nodes


# Unbalanced Transportation Model
<img src="https://github.com/amirkfard/CVL609/blob/main/img/mip_85.webp?raw=1\" width="700">
<img src="https://github.com/amirkfard/CVL609/blob/main/img/mip_86.webp?raw=1\" width="700">

In [47]:
solver = pywraplp.Solver.CreateSolver('SCIP')

infinity = solver.infinity()
x = {}
for s in Sources:
    for d in Destinations:
        x[s,d] = solver.IntVar(0, infinity, f'x[{s},{d}]')

print('Number of variables =', solver.NumVariables())

for s in Sources:
    constraint_expr = []
    for d in Destinations:
        constraint_expr.append(x[s,d])
    solver.Add(sum(constraint_expr) <= Capacities[s])

for d in Destinations:
    constraint_expr = []
    for s in Sources:
        constraint_expr.append(x[s,d])
    solver.Add(sum(constraint_expr) == Requirements[d])
    
print('Number of constraints =', solver.NumConstraints())

obj_expr = []
for s in Sources:
    for d in Destinations:
        obj_expr.append(Costs[d][s] * x[s,d])
solver.Minimize(solver.Sum(obj_expr))

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    for s in Sources:
        for d in Destinations:
            print(x[s,d].name(), ' = ', x[s,d].solution_value())
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
else:
    print('The problem does not have an optimal solution.')

Number of variables = 9
Number of constraints = 6
Objective value = 3000.0
x[W,E]  =  0.0
x[W,T]  =  100.0
x[W,O]  =  0.0
x[M,E]  =  300.0
x[M,T]  =  0.0
x[M,O]  =  0.0
x[H,E]  =  0.0
x[H,T]  =  100.0
x[H,O]  =  200.0

Problem solved in 9.000000 milliseconds
Problem solved in 7 iterations
Problem solved in 1 branch-and-bound nodes


<img src="https://github.com/amirkfard/CVL609/blob/main/img/mip_87.webp?raw=1\" width="700">
<img src="https://github.com/amirkfard/CVL609/blob/main/img/mip_88.webp?raw=1\" width="700">

In [48]:
solver = pywraplp.Solver.CreateSolver('SCIP')

infinity = solver.infinity()
x = {}
for s in Sources:
    for d in Destinations:
        x[s,d] = solver.IntVar(0, infinity, f'x[{s},{d}]')

print('Number of variables =', solver.NumVariables())

for s in Sources:
    constraint_expr = []
    for d in Destinations:
        constraint_expr.append(x[s,d])
    solver.Add(sum(constraint_expr) == Capacities[s])

for d in Destinations:
    constraint_expr = []
    for s in Sources:
        constraint_expr.append(x[s,d])
    solver.Add(sum(constraint_expr) <= Requirements[d])
    
print('Number of constraints =', solver.NumConstraints())

obj_expr = []
for s in Sources:
    for d in Destinations:
        obj_expr.append(Costs[d][s] * x[s,d])
solver.Minimize(solver.Sum(obj_expr))

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    for s in Sources:
        for d in Destinations:
            print(x[s,d].name(), ' = ', x[s,d].solution_value())
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
else:
    print('The problem does not have an optimal solution.')

Number of variables = 9
Number of constraints = 6
Objective value = 3000.0
x[W,E]  =  100.0
x[W,T]  =  0.0
x[W,O]  =  0.0
x[M,E]  =  200.0
x[M,T]  =  100.0
x[M,O]  =  0.0
x[H,E]  =  0.0
x[H,T]  =  100.0
x[H,O]  =  200.0

Problem solved in 8.000000 milliseconds
Problem solved in 5 iterations
Problem solved in 1 branch-and-bound nodes


<img src="https://github.com/amirkfard/CVL609/blob/main/img/mip_89.webp?raw=1\" width="700">
<img src="https://github.com/amirkfard/CVL609/blob/main/img/mip_810.webp?raw=1\" width="700">
<img src="https://github.com/amirkfard/CVL609/blob/main/img/mip_811.webp?raw=1\" width="700">
<img src="https://github.com/amirkfard/CVL609/blob/main/img/mip_812.webp?raw=1\" width="700">
<img src="https://github.com/amirkfard/CVL609/blob/main/img/mip_813.webp?raw=1\" width="700">

In [57]:
Sources = ['W','M','H']
Destinations = ['E','T','O','M']
Capacities = {'W' : 100, 'M' : 300, 'H' : 300}
Requirements = {'E' : 300, 'T' : 200, 'O' : 200, 'M' : 0}
Costs_Values = [[5,4,3,2],[3,2,1,0],[9,7,5,3]]
Costs = pd.DataFrame(Costs_Values , columns = Destinations , index = Sources)
print(Costs)

   E  T  O  M
W  5  4  3  2
M  3  2  1  0
H  9  7  5  3


In [62]:
solver = pywraplp.Solver.CreateSolver('SCIP')

infinity = solver.infinity()
x = {}
for s in Sources:
    for d in Destinations:
        x[s,d] = solver.IntVar(0, infinity, f'x[{s},{d}]')

print('Number of variables =', solver.NumVariables())

for s in Sources:
    constraint_expr = []
    for d in Destinations:
        constraint_expr.append(x[s,d])
    solver.Add(sum(constraint_expr) == Capacities[s])

for d in Destinations:
    constraint_expr = []
    for s in Sources:
        constraint_expr.append(x[s,d])
    solver.Add(sum(constraint_expr) == Requirements[d])

solver.Add(x['M','M'] == 0)

print('Number of constraints =', solver.NumConstraints())

obj_expr = []
for s in Sources:
    for d in Destinations:
        obj_expr.append(Costs[d][s] * x[s,d])
solver.Minimize(solver.Sum(obj_expr))

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    for s in Sources:
        for d in Destinations:
            print(x[s,d].name(), ' = ', x[s,d].solution_value())
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
else:
    print('The problem does not have an optimal solution.')

Number of variables = 12
Number of constraints = 8
Objective value = 3000.0
x[W,E]  =  100.0
x[W,T]  =  0.0
x[W,O]  =  0.0
x[W,M]  =  0.0
x[M,E]  =  200.0
x[M,T]  =  100.0
x[M,O]  =  0.0
x[M,M]  =  0.0
x[H,E]  =  0.0
x[H,T]  =  100.0
x[H,O]  =  200.0
x[H,M]  =  0.0

Problem solved in 13.000000 milliseconds
Problem solved in 6 iterations
Problem solved in 1 branch-and-bound nodes
